# This script loads the tflite model and uses it on a bunch of images


In [ ]:
import os
import tensorflow as tf
import glob
import numpy as np
import cv2 as cv

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
%pip install matplotlib

In [ ]:
# this is your labels you'v defined in 2_doAnnotations
LABELS =["Briller"]
#LABELS = ['Membran_Mangler','Stort_Tape','Lille_Tape','Kul_Mangler',]
paths ={
"PRE_TRAINED_MODEL" :  os.path.join('model','detect.tflite'),# this model is trained with 10 images and 4000 steps
"HOME_MADE_MODEL" : os.path.join('Tensorflow','workspace','models','my_ssd_mobnet','tfliteexport','saved_model','detect.tflite'),
'VALIDATION_PATH':os.path.join('Images','Validation')
}

In [ ]:
def __set_input_tensor__(image,interpreter):
    """Sets the input tensor."""
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = np.expand_dims((image - 255) / 255, axis=0)

def __get_output_tensor__(index,interpreter):
    """Returns the output tensor at the given index."""
    output_details = interpreter.get_output_details()[index]
    tensor = np.squeeze(
        interpreter.get_tensor(output_details['index']))
    return tensor
def __generate_dataframe__(scores, boxes, count, classes, threshold=0.8):
        ''' Function to generate a dataframe in the expected format by the tensorflow algorithm
            scores : array of prediction scores between 0 and 1
            boxes  : array with coordinates representing the bounding box, in a format between 0 and 1 compared to the  full image
            classes : array of classes indicated what is to what 
            threshold : float between 0 and 1 indicating how confidant we want to be
        '''
        results = []
        for i in range(count):
            if scores[i] >= threshold:
                result = {
                    'bounding_box': boxes[i],
                    'class_id': classes[i],
                    'score': scores[i]
                }
                results.append(result)
        return results
def __get_true_bounding_box__(result, camera_height, camera_width, scaling):
        ''' CAMERA_HEIGHT & CAMERA_WIDTH is the frame size in the X&Y direction, 800x600 
            result is the result of the tensorflow detection algoritme
            We expect an list containing a bounding_box as key with an np.array with the four corners 
        '''
        ymin, xmin, ymax, xmax = result['bounding_box']
        xmin = int(max(1, xmin * camera_width) * scaling)
        xmax = int(min(camera_width, xmax * camera_width) * scaling)
        ymin = int(max(1, ymin * camera_height) * scaling)
        ymax = int(min(camera_height, ymax * camera_height) * scaling)
        return ymin, xmin, ymax, xmax
def detect_objects(frame,interpreter, threshold=0.8):
        """Returns a list of detection results, each a dictionary of object info."""
        TENSORFLOW_MODEL_SIZE=(320, 320)
        # pylint: disable=no-member
        image = cv.resize(frame, TENSORFLOW_MODEL_SIZE)
        __set_input_tensor__(image,interpreter)
        interpreter.invoke()
        # Get all output details
        scores =__get_output_tensor__(0,interpreter)
        boxes = __get_output_tensor__(1,interpreter)
        count = __get_output_tensor__(2,interpreter).astype(int)
        classes = __get_output_tensor__(3,interpreter).astype(int)
        return __generate_dataframe__(scores, boxes, count, classes,
                                             threshold)
def draw_rectangle(frame, res,labels,image_size):
        ''' draw the found bounding box detected from self.detect_objects '''
        for result in res:
            (ymin, xmin, ymax, xmax) = __get_true_bounding_box__(result,image_size[0], image_size[1],1)
            # pylint: disable=no-member
            cv.rectangle(frame, (xmin, ymin), (xmax, ymax),
                            (0,0,255), 3)
            # pylint: disable=no-member
            cv.putText(frame, labels[result['class_id']],
                       (xmin, ymax),
                        cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2,
                        cv.LINE_AA)
            
        return frame

# Test your model!
In the next part we are using your model for detecting an object in the image. 


In [ ]:
# load the model
#MODEL_TO_USE = paths['HOME_MADE_MODEL']
MODEL_TO_USE = paths['PRE_TRAINED_MODEL']
print("We are using the following model <",MODEL_TO_USE,">")


## Test your model on a single image

In [ ]:
interpreter = tf.lite.Interpreter(model_path=MODEL_TO_USE)
interpreter.allocate_tensors()

#using interactive mode
#%matplotlib tk

# create a webcam object and read the image
webcam = cv.VideoCapture(0)
# make the webcam only show the latest image
webcam.set(cv.CAP_PROP_BUFFERSIZE, 1)
# read the image
_,img = webcam.read()
# print image size
image_size = img.shape
print(image_size)
# release the webcam
webcam.release()
# detect the objects
res = detect_objects(img,interpreter, threshold=0.9)
# draw the bounding box
drawing = draw_rectangle(img,res,LABELS,image_size)
# show the image
plt.figure(num=1,figsize=(10,10))
plt.imshow(cv.cvtColor(drawing, cv.COLOR_BGR2RGB))
plt.show()

In [ ]:
import time

interpreter = tf.lite.Interpreter(model_path=MODEL_TO_USE)
interpreter.allocate_tensors()

%matplotlib inline

# create a webcam object and read the image
webcam = cv.VideoCapture(0)
# make the webcam only show the latest image
webcam.set(cv.CAP_PROP_BUFFERSIZE, 1)
# read 10 images
for i in range(10):
    _,img = webcam.read()
    # print image size
    image_size = img.shape
    print(image_size)

    # detect the objects
    res = detect_objects(img,interpreter, threshold=0.9)
    # draw the bounding box
    drawing = draw_rectangle(img,res,LABELS,image_size)
    # show the image
    plt.figure(num=1,figsize=(10,10))
    plt.imshow(cv.cvtColor(drawing, cv.COLOR_BGR2RGB))
    plt.show()
    # wait 3 seconds
    time.sleep(3)

# release the webcam
webcam.release()